In [1]:
import yfinance as yf
import pandas as pd
import functions as f
import numpy as np

%load_ext autoreload
%autoreload 2

stock = yf.Ticker("SPY")
dividends = stock.dividends

In [2]:
spot_prices = stock.history(period="1y")["Close"].to_frame()
spot_price = stock.history(period="1d")["Close"].iloc[-1]

In [3]:
daily_returns = spot_prices.pct_change().dropna()

In [4]:
historical_volatility = daily_returns.std() * (252**0.5)

In [5]:
expiration_dates = stock.options

In [6]:
calls_all = [stock.option_chain(date).calls for date in expiration_dates]
puts_all = [stock.option_chain(date).puts for date in expiration_dates]

In [7]:
calls_dict = {date: stock.option_chain(date).calls for date in expiration_dates}
puts_dict = {date: stock.option_chain(date).puts for date in expiration_dates}

In [8]:
from datetime import datetime

expiration_df = pd.DataFrame({'expiration': expiration_dates})
expiration_df['time_to_expiry'] = expiration_df['expiration'].apply(
    lambda x: (datetime.strptime(x, '%Y-%m-%d') - datetime.now()).days / 365
)

In [9]:
# Add expiration column to each DataFrame in calls_dict and puts_dict
for date, df in calls_dict.items():
    df['expiration'] = date

for date, df in puts_dict.items():
    df['expiration'] = date

# Concatenate all DataFrames from calls_dict and puts_dict
calls_all = pd.concat(calls_dict.values())
puts_all = pd.concat(puts_dict.values())

# Combine calls and puts into a single DataFrame
options_data = pd.concat([calls_all, puts_all])

# Select desired columns
calls_data = calls_all[['contractSymbol', 'strike', 'lastPrice', 'impliedVolatility', 'expiration']]


In [10]:
calls_data.iloc[0]

contractSymbol       SPY241107C00400000
strike                            400.0
lastPrice                         170.8
impliedVolatility              3.398439
expiration                   2024-11-07
Name: 0, dtype: object

In [11]:
bs = f.BlackScholes(
    type=f.OptionType.PUT,
    spot=594.7899169921875,
    exercise=105,
    years=0.8931506849315068
,
    volatility=0.11209501132709629
,
    risk_free_rate=0.0442,
    dividend_yield=0
)
bs.price()

0.0

In [12]:
f.BlackScholes.find_implied_volatility(
    type=f.OptionType.CALL,
    spot=spot_price,
    exercise=calls_data.iloc[3237]["strike"],
    years=f.calculate_time_to_expiration(calls_data.iloc[3237]["expiration"]),
    target_price=calls_data.iloc[3237]["lastPrice"],
    risk_free_rate=0.0442,
    dividend_yield=0
)


0.11276133797401923

In [13]:
f.calculate_time_to_expiration(calls_data.iloc[3237]["expiration"])

0.8931506849315068

In [14]:
calls_data.iloc[3237]["strike"]

685.0

In [15]:
calls_data.iloc[3237]["lastPrice"]

6.2

In [16]:
calls_data.iloc[3237]

contractSymbol       SPY250930C00685000
strike                            685.0
lastPrice                           6.2
impliedVolatility              0.139642
expiration                   2025-09-30
Name: 122, dtype: object

In [17]:
type(calls_data)

pandas.core.frame.DataFrame

In [18]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

imp_vol_data = pd.DataFrame(columns=["ContractSymbol","StrikePrice","TimeToExpiry", "ImpliedVolatility"])
imp_vol_data.name = "ImpliedVolatility"
df_index = 0

for i in range(len(calls_data)):
    if f.calculate_time_to_expiration(calls_data.iloc[i]["expiration"]) > 0:
        imp = f.BlackScholes.find_implied_volatility(
        type=f.OptionType.CALL,
        spot=spot_price,
        exercise=calls_data.iloc[i]["strike"],
        years=f.calculate_time_to_expiration(calls_data.iloc[i]["expiration"]),
        target_price=calls_data.iloc[i]["lastPrice"],
        risk_free_rate=0.0442,
        dividend_yield=0
        )
        imp_vol_data.loc[df_index] = [calls_data.iloc[i]['contractSymbol'],calls_data.iloc[i]["strike"],\
                                calls_data.iloc[i]["expiration"],imp]
        df_index += 1           `

In [19]:
imp_vol_data = imp_vol_data.dropna()

In [20]:
imp_vol_data

,ContractSymbol,StrikePrice,TimeToExpiry,ImpliedVolatility
13,SPY241111C00560000,560.0,2024-11-11,0.289900
17,SPY241111C00564000,564.0,2024-11-11,0.339456
19,SPY241111C00566000,566.0,2024-11-11,0.429054
21,SPY241111C00568000,568.0,2024-11-11,0.369764
23,SPY241111C00570000,570.0,2024-11-11,0.266158
...,...,...,...,...
3641,SPY270115C00880000,880.0,2027-01-15,0.115205
3642,SPY270115C00885000,885.0,2027-01-15,0.113241
3643,SPY270115C00890000,890.0,2027-01-15,0.111710
3644,SPY270115C00895000,895.0,2027-01-15,0.108180


In [21]:
min_percentage = 0.7
max_percentage = 1.3

min_strike_price = spot_price * min_percentage
max_strike_price = spot_price * max_percentage

num_points = 100  # Number of points in the range
strike_prices = np.linspace(min_strike_price, max_strike_price, num_points)

In [44]:
strike_prices

array([416.92698975, 420.53674723, 424.14650472, 427.75626221,
       431.36601969, 434.97577718, 438.58553467, 442.19529215,
       445.80504964, 449.41480713, 453.02456462, 456.6343221 ,
       460.24407959, 463.85383708, 467.46359456, 471.07335205,
       474.68310954, 478.29286702, 481.90262451, 485.512382  ,
       489.12213949, 492.73189697, 496.34165446, 499.95141195,
       503.56116943, 507.17092692, 510.78068441, 514.39044189,
       518.00019938, 521.60995687, 525.21971436, 528.82947184,
       532.43922933, 536.04898682, 539.6587443 , 543.26850179,
       546.87825928, 550.48801676, 554.09777425, 557.70753174,
       561.31728923, 564.92704671, 568.5368042 , 572.14656169,
       575.75631917, 579.36607666, 582.97583415, 586.58559163,
       590.19534912, 593.80510661, 597.4148641 , 601.02462158,
       604.63437907, 608.24413656, 611.85389404, 615.46365153,
       619.07340902, 622.6831665 , 626.29292399, 629.90268148,
       633.51243896, 637.12219645, 640.73195394, 644.34